In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using CairoMakie
using Colors
using DelimitedFiles
using Distributions
using LinearAlgebra
using NPZ

In [ ]:
traj = load_traj("saved-pulses-2023-12-13/single_qubit_gateset_R1e-3.jld2");

In [ ]:
function trapezoid_rule(y, dx)
    return dx * (sum(y) + 0.5 * (y[1] + y[end]))
end

# Notice that we are "off" by a factor of π (this is be design)
println(abs(1 - trapezoid_rule(traj[:a][1, :], 0.2) / (π  + π / 2)))

# Because we made orthogonal pulses at different amplitudes in the initial condition
println(abs(1 - trapezoid_rule(traj[:a][2, :], 0.2) / (π / 2)))

# This extra factor of π holds for both gates
println(abs(1 - trapezoid_rule(traj[:a][3, :], 0.2) / (π  + π / 4)))

# After the correction, discrepancies are about the same order of magnitude
println(abs(1 - trapezoid_rule(traj[:a][4, :], 0.2) / (π / 4)))


In [ ]:
# Operators 
const n_levels = 2
at = create(n_levels)
a = annihilate(n_levels)

H_operators = Dict(
        "X" => a + at,
        "Y" => -im * (a - at),
        "Z" => I - 2 * at * a,
)

# Time
T = 50
Δt = 2/9
IBM_Δt = 2/9

qubit = QuantumSystem(zeros(2, 2), [H_operators["X"]])
;

In [ ]:
gᵢ = 1

IBM_Ũ⃗s = unitary_rollout(traj[:a][gᵢ:gᵢ, 1:end], IBM_Δt, qubit)
IBM_Us = [iso_vec_to_operator(U) for U ∈ eachcol(IBM_Ũ⃗s)]

Ũ⃗s = unitary_rollout(traj[:a][gᵢ:gᵢ, 1:end], Δt, qubit)
Us = [iso_vec_to_operator(U) for U ∈ eachcol(Ũ⃗s)]
size(Ũ⃗s)

In [ ]:
# Load python array
py_Us = npzread("../notebooks/qiskit_custom_sim_$(gᵢ-1).npy")
py_Ũ⃗s = hcat([operator_to_iso_vec(U) for U ∈ eachslice(py_Us, dims=1)]...)
size(py_Ũ⃗s)

In [ ]:
fig = Figure(resolution=(500, 300))
ax = Axis(fig[1, 1])
colors = distinguishable_colors(8)
for row in 1:8
    lines!(ax, py_Ũ⃗s[row, :], color=colors[row])
    # lines!(ax, Ũ⃗s[row, :], color=colors[row], linestyle=:dash)
    lines!(ax, IBM_Ũ⃗s[row, :], color=colors[row], linestyle=:dashdot)
end
fig

We could fix this discrepancy with ILC.

The numerical integration is not the same in Qiskit. Why do they use a continuous integration strategy for a PWC control pulse?